In [1]:
#from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
#import matplotlib.pyplot as plt
import numpy as np
from tensorboardX import SummaryWriter

In [6]:
pip install tensorboardX

     |████████████████████████████████| 120 kB 628 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Load train set and test set and normalize the images in range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(( 0.1307,), ( 0.3081,))])

#50000 images training
trainset = torchvision.datasets.MNIST(root='./mnist', train=True,
                                        download=True, transform=transform)
#We load 4 samples per batch reduce the traininset to 12500
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

#print(len(trainset))
#10000 images test
testset = torchvision.datasets.MNIST(root='./mnist', train=False,
                                       download=True, transform=transform)
#We load 4 samples per batchreduce the traininset to 2500
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)


classes = list(range(10))

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )

        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)

        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform the usual forward pass
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net()

In [4]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
#Training
model.train()
total = 0 
correct = 0
running_loss = 0
for epoch in range(1, 1 + 1): 
    model.train()
    writer = SummaryWriter(logdir='TransfNet_MNIST/training_epoch%d'%(epoch))
    for i, datas in enumerate(trainloader):
        inputs, labels = datas

        optimizer.zero_grad()
        output = model(inputs)
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
        _,predicted=torch.max(output,1)
        running_loss += loss.item()
        total += labels.size(0)#count the number of labels with right shape
        correct += (predicted == labels).sum().item()
        if i % 500 == 0:
            print('[%d, %5d] loss: %.3f, accuracy: %.3f' %
                  (epoch , i + 1, running_loss / 2000, 100*float(correct)/float(total)))
            writer.add_scalar('accuracy_train', 100*float(correct)/float(total), i)
            writer.add_scalar('loss_train', running_loss / 500, i)
            running_loss = 0.0
            correct = 0
            total = 0
    writer.close()
print ('Finished Training')

[1,     1] loss: 0.001, accuracy: 0.000
[1,   501] loss: 0.575, accuracy: 11.450
[1,  1001] loss: 0.575, accuracy: 11.300
[1,  1501] loss: 0.576, accuracy: 10.300
[1,  2001] loss: 0.575, accuracy: 11.700
[1,  2501] loss: 0.575, accuracy: 11.900
[1,  3001] loss: 0.576, accuracy: 11.450
[1,  3501] loss: 0.575, accuracy: 11.250
[1,  4001] loss: 0.575, accuracy: 11.750
[1,  4501] loss: 0.575, accuracy: 11.350
[1,  5001] loss: 0.575, accuracy: 11.650
[1,  5501] loss: 0.575, accuracy: 11.500
[1,  6001] loss: 0.576, accuracy: 10.350
[1,  6501] loss: 0.576, accuracy: 10.900
[1,  7001] loss: 0.575, accuracy: 11.850
[1,  7501] loss: 0.574, accuracy: 11.000
[1,  8001] loss: 0.576, accuracy: 11.150
[1,  8501] loss: 0.576, accuracy: 12.050
[1,  9001] loss: 0.576, accuracy: 10.700
[1,  9501] loss: 0.576, accuracy: 9.800
[1, 10001] loss: 0.575, accuracy: 10.850
[1, 10501] loss: 0.575, accuracy: 11.900
[1, 11001] loss: 0.576, accuracy: 10.500
[1, 11501] loss: 0.575, accuracy: 11.850
[1, 12001] loss: 0

In [9]:
#training saving
PATH = './MNIST_TransformerNet_16epochs.pth'
torch.save(model.state_dict(), PATH)

In [11]:
PATH = './MNIST_TransformerNet_16epochs.pth'
#Load previously net from choosen training
model.load_state_dict(torch.load(PATH))
writer = SummaryWriter(logdir='TransfNet_MNIST/test')
with torch.no_grad():#desactivate autograd engine ( used to perform validation )    
    model.eval()
    test_loss = 0
    correct = 0
    correct_final = 0
    total = 0
    total_final = 0
    
    for i, datas in enumerate(testloader):
        inputs, labels = datas
        output = model(inputs)
        loss = F.nll_loss(output, labels, size_average=False)
        # sum up batch loss
        test_loss += loss.item()
        # get the index of the max log-probability
        pred = output.max(1, keepdim=True)[1]
        total += labels.size(0) #count the number of labels with right shape
        total_final += labels.size(0) #count the number of labels with right shape
        correct += (pred == labels).sum().item() #count the number of right labels
        correct_final += (pred == labels).sum().item() #count the number of right labels
        test_loss += loss.item()
        #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%5d] loss: %.3f, accuracy: %.3f' %( i + 1, (test_loss / 2000), (100*correct/total) ))
        
            writer.add_scalar('accuracy_test', 100*correct/total, i)
            writer.add_scalar('loss_test', test_loss / 2000, i)
        
            test_loss = 0.0
            correct = 0
            total = 0

writer.close()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct_final / total_final))

/Users/valentinibars/opt/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[  500] loss: 4.603, accuracy: 46.800
[ 1000] loss: 4.605, accuracy: 43.200
[ 1500] loss: 4.603, accuracy: 47.200
[ 2000] loss: 4.605, accuracy: 43.800
[ 2500] loss: 4.601, accuracy: 46.000
Accuracy of the network on the 10000 test images: 45 %
